In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Snark Tutorial").setMaster("local")
sc = SparkContext(conf=conf)

In [3]:
from util import pubtator_doc_generator
raw_docs = []
for d in pubtator_doc_generator('bioconcepts2pubtator_offsets.dev'):
    raw_docs.append(d)

In [4]:
len(raw_docs)

1152

In [5]:
raw_docs_rdd = sc.parallelize(raw_docs)